# 3. Weighted K Means

We need to weigh/normalize based on the dimensions of the image, the amplitude of the signal, and the number of frames (time steps).

We want to introduce a factor that lets us weigh the spatial indexes more than the signals.


## 2-Dimensional ROIs

In [2]:
PIXELS_PER_CM = 8

In [3]:
import numpy as np
import pandas as pd
from collections import namedtuple
Dimensions = namedtuple('Dimensions', ['width', 'height'])
Point = namedtuple('Point', ['x', 'y'])

class ROI:
    """Represents a region of interest (ROI). This is a rectangle which specifies a certain region of an image."""
    WIDTH_PIXELS: int = None  # the width of all ROIs in pixels
    HEIGHT_PIXELS: int = None  # the height of all ROIs in pixels

    N_HORIZONTAL: int = None  # the number of ROIs along the x-axis
    N_VERTICAL: int = None  # the number of ROIs along the y-axis

    @classmethod
    @property
    def width_cm(cls) -> float:
        """The width in cm"""
        return cls.WIDTH_PIXELS / PIXELS_PER_CM

    @classmethod
    @property
    def height_cm(cls) -> float:
        """The height in cm"""
        return cls.HEIGHT_PIXELS / PIXELS_PER_CM

    def __init__(self, x_idx, y_idx):
        self.x_idx = x_idx  # the x index on the grid of ROIs
        self.y_idx = y_idx

    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        return f'ROI({self.x_idx};{self.y_idx})'

    def corners_pixels(self) -> tuple[Point, Point]:
        """Returns the upper left and lower right corners (in pixels) of the ROI as Point objects. The corner points and edges \
        should be *included* in the ROI"""
        x_ul = self.x_idx * ROI.WIDTH_PIXELS  # upper left x
        y_ul = self.y_idx * ROI.HEIGHT_PIXELS  # upper left y
        x_lr = x_ul + ROI.WIDTH_PIXELS - 1  # lower right x
        y_lr = y_ul + ROI.HEIGHT_PIXELS - 1  # lower right y

        return Point(x_ul, y_ul), Point(x_lr, y_lr)

    def center_pixels(self) -> Point:
        """Returns the center of the ROI as a Point object."""
        x = self.x_idx * ROI.WIDTH_PIXELS + ROI.WIDTH_PIXELS / 2 - 0.5
        y = self.y_idx * ROI.HEIGHT_PIXELS + ROI.HEIGHT_PIXELS / 2 - 0.5
        return Point(x, y)

    def corners_cm(self) -> Point:
        """Returns the upper left and lower right corners (in cm) of the ROI as Point objects."""
        # upper left corner:
        x_ul = self.x_idx * ROI.width_cm
        y_ul = self.y_idx * ROI.height_cm
        # lower right corner:
        x_lr = x_ul + ROI.width_cm
        y_lr = y_ul + ROI.height_cm
        return Point(x_ul, y_ul), Point(x_lr, y_lr)

    def center_cm(self) -> Point:
        """Returns the center of the ROI in cm."""
        x_idx_center = self.x_idx + 0.5
        y_idx_center = self.y_idx + 0.5

        return Point(
            x_idx_center * ROI.width_cm,
            y_idx_center * ROI.height_cm
        )

In [4]:
# Vars
spatial_weight = 2
img_dims = Dimensions(16, 16)
n_frames = 20
ROI.WIDTH_PIXELS = 8
ROI.HEIGHT_PIXELS = 4
ROI.N_HORIZONTAL = img_dims.width // ROI.WIDTH_PIXELS
ROI.N_VERTICAL = img_dims.height // ROI.HEIGHT_PIXELS
print(f'{ROI.N_HORIZONTAL=}, {ROI.N_VERTICAL=}')

ROI.N_HORIZONTAL=2, ROI.N_VERTICAL=4


In [5]:
# generate Data
rois = [ROI(x, y) for x in range(ROI.N_HORIZONTAL) for y in range(ROI.N_VERTICAL)]

signals_df = pd.DataFrame(
    {roi: np.random.uniform(low=-50, high=50, size=n_frames) for roi in rois}
)

roi_centers = pd.DataFrame(
    {roi: [roi.center_cm().x, roi.center_cm().y] for roi in rois},
    index=['x_center_cm', 'y_center_cm']
)

In [6]:
# Normalizing
max_amplitude_overall = signals_df.abs().max().max()
normalized_signals = signals_df / max_amplitude_overall

# adjust spatial indexes for n_frames and scale by spatial weight
roi_centers *= np.sqrt(n_frames) * spatial_weight

signals_indexes = pd.concat([roi_centers, normalized_signals])
signals_indexes.index = signals_indexes.index.astype(str)

In [7]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4, n_init='auto')
kmeans.fit(signals_indexes.T)

KMeans(n_clusters=4, n_init='auto')

In [8]:
kmeans.labels_

array([3, 1, 1, 1, 0, 0, 2, 2], dtype=int32)